# 製品価格を予測します

## 3日目：トレーニング！

＃重要です私を読んでください!!

以下のPIPインストールを実行すると、FSSPECの互換性のないバージョンについて不平を言うPIPからエラーが発生する可能性があります。

そのエラーを無視する必要があります！ FSSPECのバージョンは、Huggingfaceが必要とする適切なバージョンです。

ChatGptに尋ねると、FSSPECのより最近のバージョンをインストールすることをお勧めします。しかし、それは問題があるでしょう。 Huggingfaceは、ファイルシステムに関するあいまいなエラーでデータセットを後でロードできません。

したがって、PIPインストールは以下に表示されているように実行してください。エラーが発生した場合は、逆の方を見てください。

In [ ]:
# ピップインストール

!pip install -q --upgrade torch==2.5.1+cu124 torchvision==0.20.1+cu124 torchaudio==2.5.1+cu124 --index-url https://download.pytorch.org/whl/cu124
!pip install -q --upgrade requests==2.32.3 bitsandbytes==0.46.0 transformers==4.48.3 accelerate==1.3.0 datasets==3.2.0 peft==0.14.0 trl==0.14.0 matplotlib wandb

In [ ]:
# 輸入
# イスラム教S.に感謝します。

import os
import re
import math
from tqdm import tqdm
from google.colab import userdata
from huggingface_hub import login
import torch
import transformers
from transformers import AutoModelForCausalLM, AutoTokenizer, TrainingArguments, set_seed, BitsAndBytesConfig
from datasets import load_dataset, Dataset, DatasetDict
import wandb
from peft import LoraConfig
from trl import SFTTrainer, SFTConfig
from datetime import datetime
import matplotlib.pyplot as plt

In [ ]:
# 定数

BASE_MODEL = "meta-llama/Meta-Llama-3.1-8B"
PROJECT_NAME = "pricer"
HF_USER = "ed-donner" # ここにHFの名前！

# データ

DATASET_NAME = f"{HF_USER}/pricer-data"
# または、アップロードしたものを使用してください
# dataset_name = "ed-donner/pricer-data"
MAX_SEQUENCE_LENGTH = 182

# ハブでモデルを保存するために名前を実行します

RUN_NAME =  f"{datetime.now():%Y-%m-%d_%H.%M.%S}"
PROJECT_RUN_NAME = f"{PROJECT_NAME}-{RUN_NAME}"
HUB_MODEL_NAME = f"{HF_USER}/{PROJECT_RUN_NAME}"

# クロラのハイパーパラメーター

LORA_R = 32
LORA_ALPHA = 64
TARGET_MODULES = ["q_proj", "v_proj", "k_proj", "o_proj"]
LORA_DROPOUT = 0.1
QUANT_4_BIT = True

# トレーニング用のハイパーパラメーター

EPOCHS = 1 # 必要に応じてより多くのエポックを行うことができますが、1つだけが必要です - おそらくそれ以上はやり過ぎです
BATCH_SIZE = 4 # A100ボックスでこれは16までになります
GRADIENT_ACCUMULATION_STEPS = 1
LEARNING_RATE = 1e-4
LR_SCHEDULER_TYPE = 'cosine'
WARMUP_RATIO = 0.03
OPTIMIZER = "paged_adamw_32bit"

# admin config- save_stepsはハブにアップロードする頻度であることに注意してください
# 私はこれを5000から2000に変更したので、より頻繁にセーブするようになりました

STEPS = 50
SAVE_STEPS = 2000
LOG_TO_WANDB = True

%matplotlib inline

In [ ]:
HUB_MODEL_NAME

＃オプティマイザーの詳細

https://huggingface.co/docs/transformers/main/en/perf_train_gpu_one#optimizer-choice

最も一般的なのは、アダムまたはアダム（体重減衰のあるアダム）です。  
アダムは、以前の勾配のローリング平均を保存することにより、良好な収束を達成します。ただし、モデルパラメーターの数の順序の追加メモリフットプリントを追加します。


### ログインして、フェイスと重みとバイアスを抱きしめます

まだHuggingfaceアカウントをお持ちでない場合は、https：//huggingface.coにアクセスしてサインアップしてトークンを作成します。

次に、左のキーアイコンをクリックして、このノートブックの秘密を選択し、トークンとして値を持つ「hf_token」と呼ばれる新しい秘密を追加します。

https://wandb.aiでweightsandbiaseについてこれを繰り返し、 `wandb_api_key`と呼ばれる秘密を追加します

In [ ]:
# Huggingfaceにログインします

hf_token = userdata.get('HF_TOKEN')
login(hf_token, add_to_git_credential=True)

In [ ]:
# ウェイトとバイアスにログインします
wandb_api_key = userdata.get('WANDB_API_KEY')
os.environ["WANDB_API_KEY"] = wandb_api_key
wandb.login()

# プロジェクトに対して記録するように重みとバイアスを構成します
os.environ["WANDB_PROJECT"] = PROJECT_NAME
os.environ["WANDB_LOG_MODEL"] = "checkpoint" if LOG_TO_WANDB else "end"
os.environ["WANDB_WATCH"] = "gradients"

In [ ]:
dataset = load_dataset(DATASET_NAME)
train = dataset['train']
test = dataset['test']

In [ ]:
# 代わりにトレーニングデータセットを20,000ポイントに削減したい場合は、この行を解除します。
# Train = Train.Select（Range（20000））

In [ ]:
if LOG_TO_WANDB:
  wandb.init(project=PROJECT_NAME, name=RUN_NAME)

## 次に、トークン剤とモデルをロードします

モデルは「量子化」されています - 精度を4ビットに減らしています。

In [ ]:
# 適切な量子化を選択します

if QUANT_4_BIT:
  quant_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_compute_dtype=torch.bfloat16,
    bnb_4bit_quant_type="nf4"
  )
else:
  quant_config = BitsAndBytesConfig(
    load_in_8bit=True,
    bnb_8bit_compute_dtype=torch.bfloat16
  )

In [ ]:
# トークン剤とモデルをロードします

tokenizer = AutoTokenizer.from_pretrained(BASE_MODEL, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

base_model = AutoModelForCausalLM.from_pretrained(
    BASE_MODEL,
    quantization_config=quant_config,
    device_map="auto",
)
base_model.generation_config.pad_token_id = tokenizer.pad_token_id

print(f"Memory footprint: {base_model.get_memory_footprint() / 1e6:.1f} MB")

# データコラーター

トレーニング中に、製品の説明を予測するためにモデルを訓練しようとしていないことを保証することが重要です。彼らの価格だけ。

「価格は$」までのすべてがモデルにコンテキストを提供して次のトークンを予測するためにあるが、学習する必要はないことをトレーナーに伝える必要があります。

トレーナーは、「価格は$」の後にトークンを予測するようにモデルに教える必要があります。

マスクを設定することでこれを行うための複雑な方法がありますが、幸運なことに、Huggingfaceは私たちのためにこれを大事にするための非常にシンプルなヘルパークラスを提供します。

In [ ]:
from trl import DataCollatorForCompletionOnlyLM
response_template = "Price is $"
collator = DataCollatorForCompletionOnlyLM(response_template, tokenizer=tokenizer)

# そして今

##トレーニング用の構成を設定します

2つのオブジェクトを作成する必要があります。

Loraのハイパーパラメーターを備えたLoraconfigオブジェクト

全体的なトレーニングパラメーターを備えたSFTCONFIG

In [ ]:
# まず、LORAの構成パラメーターを指定します

lora_parameters = LoraConfig(
    lora_alpha=LORA_ALPHA,
    lora_dropout=LORA_DROPOUT,
    r=LORA_R,
    bias="none",
    task_type="CAUSAL_LM",
    target_modules=TARGET_MODULES,
)

# 次に、トレーニング用の一般的な構成パラメーターを指定します

train_parameters = SFTConfig(
    output_dir=PROJECT_RUN_NAME,
    num_train_epochs=EPOCHS,
    per_device_train_batch_size=BATCH_SIZE,
    per_device_eval_batch_size=1,
    eval_strategy="no",
    gradient_accumulation_steps=GRADIENT_ACCUMULATION_STEPS,
    optim=OPTIMIZER,
    save_steps=SAVE_STEPS,
    save_total_limit=10,
    logging_steps=STEPS,
    learning_rate=LEARNING_RATE,
    weight_decay=0.001,
    fp16=False,
    bf16=True,
    max_grad_norm=0.3,
    max_steps=-1,
    warmup_ratio=WARMUP_RATIO,
    group_by_length=True,
    lr_scheduler_type=LR_SCHEDULER_TYPE,
    report_to="wandb" if LOG_TO_WANDB else None,
    run_name=RUN_NAME,
    max_seq_length=MAX_SEQUENCE_LENGTH,
    dataset_text_field="text",
    save_strategy="steps",
    hub_strategy="every_save",
    push_to_hub=True,
    hub_model_id=HUB_MODEL_NAME,
    hub_private_repo=True
)

# そして今、監督された微調整トレーナーが微調整を実行します
# これらの2セットの構成パラメーターが与えられます
# TRLの最新バージョンは、ラベルに関する警告を示しています - この警告を無視してください
# しかし、良いトレーニング結果が見られない場合はお知らせください（損失が下がっています）。

fine_tuning = SFTTrainer(
    model=base_model,
    train_dataset=train,
    peft_config=lora_parameters,
    args=train_parameters,
    data_collator=collator
  )

## 次のセルでは、微調整を開始します！

これはしばらくの間実行され、すべてのSave_Stepsステップをハブにアップロードします。

しばらくすると、Googleはあなたのcolabを止めるかもしれません。無料プランを使用している人にとって、Googleがリソースが少ないときはいつでも発生する可能性があります。有料プランの場合は誰でも、最大24時間を与えることができますが、保証はありません。

サーバーが停止した場合は、ここで私のコラブをたどって最後の保存から再開できます。

https://colab.research.google.com/drive/1qgtdvias_vwoby4uvi2vwsu0thxy8omo#scrollto = r_o04fkxmmt-

このコラブを出力での最終実行で保存したので、例を見ることができます。トリックは、fine_tunedモデルをロードするときに「is_trainable」を設定する必要があることです。

とにかく###それを念頭に置いて、これをキックオフしましょう！

In [ ]:
# 微調整！
fine_tuning.train()

# 微調整されたモデルを顔に抱きしめます
fine_tuning.model.push_to_hub(PROJECT_RUN_NAME, private=True)
print(f"Saved to the hub: {PROJECT_RUN_NAME}")

In [ ]:
if LOG_TO_WANDB:
  wandb.finish()